# Exp14. 프로젝트 - Movielens 영화 추천 실습
활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터다. 
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. 
- 별점 데이터는 대표적인 explicit 데이터지만 implicit 데이터로 간주하고 테스트할 수 있다. 
- 별점을 시청횟수로 해석해서 생각하자. 
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정한다.  


### 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남긴다. 
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 로드
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
data = pd.merge(ratings, movies)

- ratings와 movies에 모두 movie_id가 있기 때문에 이를 기준으로 두 데이터프레임을 합쳤다(merge). 

In [7]:
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
using_cols = ['user_id', 'title', 'counts']
data = data[using_cols]
data.head(10)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


- 필요한 컬럼만 남기고 제거했다. 

### 2) 데이터 탐색

In [9]:
# data에 있는 유니크한 영화 개수
data['title'].nunique()

3628

In [10]:
# data에 있는 유니크한 사용자 수
data['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개(인기순)
title_count = data.groupby('title')['user_id'].count()
title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 사용자 별 몇 개의 영화를 봤는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

### 3) 내가 선호하는 영화 5가지 골라서 data에 추가

In [13]:
my_favorite = ['Toy Story (1995)' , 'Matrix, The (1999)' ,'Terminator 2: Judgment Day (1991)' ,'Terminator, The (1984)' ,'Jurassic Park (1993)']

# 'jaehyun'이라는 user_id가 위 영화를 5번씩 봤다고 가정 (평점을 횟수로 가정했으므로)
my_playlist = pd.DataFrame({'user_id':['jaehyun']*5, 'title':my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['jaehyun']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면 위에 임의로 만든 데이터 추가
    data = data.append(my_playlist)
    
data.tail(10)  # 잘 추가되었는지 확인

,user_id,title,counts
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4
836477,5948,Identification of a Woman (Identificazione di ...,5
0,jaehyun,Toy Story (1995),5
1,jaehyun,"Matrix, The (1999)",5
2,jaehyun,Terminator 2: Judgment Day (1991),5
3,jaehyun,"Terminator, The (1984)",5
4,jaehyun,Jurassic Park (1993),5


In [14]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing하는 코드
user_to_idx = {v:k for k, v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jaehyun'])  # 6040명의 유저 중 마지막으로 추가된 유저라서 6039이 나와야 한다. 
print(title_to_idx['Toy Story (1995)'])

6039
40


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해보자. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):  # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')
    
# title_to_idx를 통해 artist 컬럼도 동일한 방식으로 인덱싱
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,40,5
1,6039,124,5
2,6039,92,5
3,6039,200,5


### 4) CSR matrix 생성

In [17]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) MF 모델 학습하기
- `implicit` 패키지는 암묵적 데이터셋을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지다. 
- 이 패키지에 구현된 `als(AlternatingLeastSquares)` 모델을 사용해보자. MF에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한 쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 als방식이 효과적인 것으로 알려져 있다. 

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분으로, 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

- AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보자. 
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미다. 데이터를 몇 번 반복해서 학습할 것인지  

- 1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다. 

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [20]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

- 모델 학습이 끝났으므로 2가지 사항을 살펴보자. 
1. jaehyun 벡터와 Toy Story (1995)의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [22]:
jaehyun, toy_story_1995 = user_to_idx['jaehyun'], title_to_idx['Toy Story (1995)']
jaehyun_vector, toy_story_1995_vector = als_model.user_factors[jaehyun], als_model.item_factors[toy_story_1995]

In [23]:
jaehyun_vector

array([-1.2295958 , -0.33232802, -0.1341222 , -0.51456434,  0.446458  ,
        0.07728854,  0.19096981,  0.65385157, -0.82453454,  0.71519434,
       -0.12251413, -0.11275071,  0.36444226,  0.50490475,  0.6683688 ,
       -0.32655227, -0.6728536 ,  1.1542741 ,  0.04124462, -0.77027273,
       -0.02716425,  0.1099931 , -0.6147232 ,  0.0828746 ,  0.64688456,
        0.35055864,  0.3766285 ,  0.23233193,  0.06284525, -0.23516467,
        0.5978214 , -0.6419357 , -0.28120074, -0.15911227,  0.39324695,
       -0.43254915,  0.16430049,  0.48797932,  0.46321246, -0.3702417 ,
       -0.15652081, -0.44176415,  0.13537045,  0.06016988,  0.7090349 ,
       -0.41191006,  0.03849759, -0.04041364,  1.3893045 ,  0.48429835,
        0.09625688,  0.71751046, -0.59683144,  0.5296188 , -0.5248479 ,
       -1.3163904 ,  0.76440775,  0.48095474, -0.551583  ,  0.77972794,
       -0.05015115, -0.44759622, -0.14093708,  0.18343073, -0.3837355 ,
       -0.16270131, -0.747808  ,  0.3343634 , -0.8574335 , -0.36

In [24]:
toy_story_1995_vector

array([-0.03450963,  0.02754663, -0.02215889, -0.01894817,  0.01973913,
        0.04939866,  0.00467446, -0.00506493,  0.00509344,  0.0312742 ,
       -0.00685987, -0.01076658,  0.00126015,  0.02765311, -0.01722812,
       -0.01300864,  0.00076677,  0.04138676,  0.00355085,  0.00894081,
       -0.00299128,  0.03839055, -0.02913265, -0.004687  ,  0.01742247,
        0.03866908, -0.01784724,  0.00717229, -0.00098376, -0.00197393,
       -0.00039604,  0.00240511,  0.02697414, -0.01266019,  0.02379697,
       -0.02108619,  0.02743129,  0.01960642,  0.0238733 , -0.01354972,
       -0.03264422,  0.02450228,  0.0167331 , -0.02692762,  0.03442114,
       -0.00638786,  0.02010877, -0.00185093,  0.04940748,  0.0099962 ,
        0.01840177,  0.01714633,  0.01856813,  0.01280127,  0.00913653,
       -0.01288868, -0.00680634,  0.00517803, -0.00948341, -0.01136119,
       -0.00943058, -0.00313953,  0.0292283 ,  0.00972348, -0.01435679,
        0.00336093,  0.01338369,  0.01476686, -0.02647845, -0.02

In [25]:
# jaehyun과 toy_story_1995를 내적하는 코드
np.dot(jaehyun_vector, toy_story_1995_vector)

0.4749143

- 1이 나와야 할 것 같은데 한참 낮은 수치가 나왔다. factors를 늘리거나 iterations를 늘려야 할 것 같다.
- 이 두 수치를 늘려 1에 가깝게 나왔다하더라도 이 모델이 잘 학습되었다고는 확신할 수 없다. 
- 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한 검증이 이루어지지 않았기 때문이다. 

In [26]:
schindler = title_to_idx['Schindler\'s List (1993)']
schindler_vector = als_model.item_factors[schindler]
np.dot(jaehyun_vector, schindler_vector)

0.0069319387

- 그렇다면 한 번도 본 적 없는 Schindler's List (1993)에 대한 예측을 어떻게 해석해야 할까? 
- 사용자와 영화 벡터 내적이 0.5를 기준으로 그보다 높으면 모델이 선호한다고 판단했다고 할 수 있을까?
- 모델은 만드는 사람이 정하기 나름이라고 한다. 객관적인 지표를 만들어 기준을 정할 수도, 도메인 경험을 통해 정할 수도 있고 한다. (도메인 경험이라니.. 오..) 
- 추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야다. (사람을 도통 알 수 없기 때문에...) 

### 7) 내가 좋아하는 영화와 비슷한 영화 추천받기
- `AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통해 비슷한 영화를 찾는다.

In [27]:
favorite_title = 'Toy Story (1995)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(40, 1.0),
 (50, 0.79038316),
 (33, 0.58450556),
 (322, 0.57963395),
 (4, 0.56650674),
 (110, 0.5385301),
 (330, 0.46631473),
 (10, 0.43585706),
 (20, 0.43047625),
 (160, 0.4059132),
 (255, 0.38720405),
 (32, 0.35368198),
 (34, 0.3503908),
 (126, 0.32902256),
 (16, 0.3266439)]

- (아티스트의 id, 유사도)를 튜플로 반환하고 있다. 이를 다시 아티스트의 이름으로 매핑하자. 

In [28]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 'Forrest Gump (1994)',
 "There's Something About Mary (1998)",
 'Hercules (1997)',
 'Mulan (1998)',
 'Shakespeare in Love (1998)',
 'Tarzan (1999)']

- 오... 토이 스토리 1편을 좋아한다고 했더니 유사한 영화로 토이 스토리 2편과 알라딘, 라이온 킹, 미녀와 야수 등 내가 좋아하는 디즈니 영화를 추천해준다. 
- iteration을 20으로만 학습했는데도 이런 결과가 나오다니 놀랍다. 
- 몇 번 더 반복하기 위해서 위의 코드를 함수화하자. 

In [29]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드 함수화
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [30]:
get_similar_title('Terminator 2: Judgment Day (1991)')

['Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Men in Black (1997)',
 'Terminator, The (1984)',
 'Braveheart (1995)',
 'Fugitive, The (1993)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Hunt for Red October, The (1990)']

- 터미네이터 2를 좋아한다고 했더니 비슷한 SF 장르의 매트릭스, 쥬라기 공원, 맨 인 블랙 등을 추천해준다. 오...

### 8) 내가 좋아할만한 영화 추천받기
- 마찬가지로 `AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통해 좋아할만한 영화를 추천받는다. 
- `filter_already_liked_items`는 유저가 이미 평가한 아이템은 제외하는 Argument다. 

In [31]:
user = user_to_idx['jaehyun']
# recommend에서는 user*item CSR Matrix를 받는다. 
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(175, 0.52369),
 (62, 0.5060674),
 (141, 0.3838625),
 (50, 0.3625343),
 (193, 0.28902286),
 (651, 0.28434318),
 (87, 0.269706),
 (38, 0.266284),
 (145, 0.263461),
 (44, 0.26289064),
 (22, 0.25830975),
 (317, 0.24959642),
 (117, 0.2443867),
 (75, 0.23911306),
 (110, 0.23157875),
 (64, 0.22202893),
 (160, 0.22021887),
 (33, 0.21436659),
 (685, 0.21374087),
 (4, 0.20895371)]

In [32]:
[idx_to_title[i[0]] for i in title_recommended]

['Men in Black (1997)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Toy Story 2 (1999)',
 'Alien (1979)',
 'Aliens (1986)',
 'Braveheart (1995)',
 'Sixth Sense, The (1999)',
 'Fifth Element, The (1997)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Back to the Future (1985)',
 'Twelve Monkeys (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Hunt for Red October, The (1990)',
 'Groundhog Day (1993)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Forrest Gump (1994)',
 'Aladdin (1992)',
 'Planet of the Apes (1968)',
 "Bug's Life, A (1998)"]

- `explain` 메서드로 이 추천에 기여한 정도를 확인할 수 있다. 

In [33]:
men_in_black = title_to_idx['Men in Black (1997)']
explain = als_model.explain(user, csr_data, itemid=men_in_black)

In [34]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Jurassic Park (1993)', 0.2987027463523288),
 ('Terminator 2: Judgment Day (1991)', 0.16590494240730191),
 ('Matrix, The (1999)', 0.0854461558579209),
 ('Toy Story (1995)', -0.006612910632082382),
 ('Terminator, The (1984)', -0.027657065635481634)]

### 회고
- 추천 시스템은 어제 Fundamentals 노드로 처음 접했는데, 다음 날인 오늘 바로 추천 시스템 프로젝트가 나와서 MF 모델에 대해 좀 더 공부할 수 있었다. 
- 데이터 전처리나 모델링 등 전체적인 흐름은 노드의 그것을 따라갔는데, iteration을 20으로 꽤나 간단하고 빠르게 학습한 것 치고는 내가 좋아할만한 영화나 내가 선호하는 5가지 영화에 기반해 비슷한 영화를 추천해주는 것이 꽤나 취향과 비슷해서 신기했다. 
- 이번 프로젝트는 explicit한 별점 데이터를 implicit한 데이터로 간주하고 테스트했는데, 실제로는 explicit한 데이터를 모으는 것보다 implicit한 데이터를 모으는 것이 더 수월하지 않나싶다. 나만 하더라도 영화를 엄청 좋아하고 봤던 영화를 여러 번 볼 정도로 영화를 즐겨 보지만 정작 넷플릭스나 왓챠 같은 OTT 플랫폼에서 내가 본 영화에 대해 평가를 한 적은 거의 없는 것 같다. 그래서...? 뭔가 사용자 본인도 모르게 암묵적으로 제공되는 데이터를 기반으로 추천을 하는게 데이터 수집 면에서??? 좀 더 수월하지 않을까 싶다. 
- 여기선 영화의 장르나 감독 등 다른 요인들에 대해서는 다루지 않았지만 이러저러한 추가적인 요인들에 기반해 추천을 한다면 좀 더 추천의 정확도를 높일 수 있지 않나 생각한다.